<a href="https://colab.research.google.com/github/RaffayScriptSlinger/Aict-Project/blob/main/MultiModal_Live_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Installation**

In [ ]:
%pip install -q -U "google-genai>=1.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 5.2 MB/s eta 0:00:00


### **Initialization**

In [13]:
from google.colab import userdata

# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_1')

In [14]:
import asyncio
import contextlib
import json
import wave

from IPython import display

from google import genai
from google.genai import types



client = genai.Client(api_key=GOOGLE_API_KEY)

In [15]:
MODEL_ID="gemini-2.0-flash-exp" # @param ["gemini-2.0-flash","gemini-2.0-flash-exp","gemini-1.5-pro","gemini-2.5-flash-preview-04-17", "gemini-2.5-pro-preview-05-06", "gemini-2.0-flash-exp-image-generation"] {"allow-input":true, isTemplate: true}

### **Helper Functions**

In [17]:
# Making Directory of Functions to Simulate Real World API Integration
def get_current_weather(args):
    # In a real app, this would query a Open Weather API
    return {
              "temperature": '33C',
              "comments": f"""The current weather of {args['location']} is Sunny and there is no chance of rain today."""
            }

def check_calendar_api(args):
    # In a real app, this would query a calendar API to check availability
    return {"available": True}

def create_meeting_api(args):
    # In a real app, this would call a calendar API to create an event
    return {"meeting_id": "123456", "created": True}

def send_email_api(args):
    # In a real app, this would call an email API
    return {"sent": True}


AVAILABLE_FUNCTIONS = {
    "get_current_weather" : get_current_weather,
    "check_calendar": check_calendar_api,
    "create_meeting": create_meeting_api,
    "send_email": send_email_api
}

In [18]:
def handle_server_content(server_content):
  model_turn = server_content.model_turn
  if model_turn:
    for part in model_turn.parts:
      executable_code = part.executable_code
      if executable_code is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'``` python\n{executable_code.code}\n```'))
        display.display(display.Markdown('-------------------------------'))

      code_execution_result = part.code_execution_result
      if code_execution_result is not None:
        display.display(display.Markdown('-------------------------------'))
        display.display(display.Markdown(f'```{code_execution_result.output}```'))
        display.display(display.Markdown('-------------------------------'))

  grounding_metadata = getattr(server_content, 'grounding_metadata', None)
  if grounding_metadata is not None:
    display.display(
        display.HTML(grounding_metadata.search_entry_point.rendered_content))

  return

In [19]:
async def handle_tool_call(session, tool_call):
  function_responses = []
  for fc in tool_call.function_calls:
     if fc.name in AVAILABLE_FUNCTIONS:
          function_to_call = AVAILABLE_FUNCTIONS[fc.name]
          response_data = function_to_call(fc.args)
     else:
          # Handle unknown functions
          response_data = {"reslt": "ok"}
          print(f"✅ Function result: {response_data}")

  function_response = types.FunctionResponse(
          id=fc.id,
          name=fc.name,
          response=response_data,  # Use actual function result
  )
  function_responses.append(function_response)
  print('\n Response >>> ', function_responses)
  await session.send_tool_response(function_responses=function_responses)
  return function_responses

In [20]:
async def run(prompt, modality="TEXT", tools=[]):
  config = {
          "tools": tools,
          "response_modalities": [modality]
  }

  async with client.aio.live.connect(model=MODEL_ID, config=config) as session:
    display.display(display.Markdown(prompt))
    display.display(display.Markdown('-------------------------------'))
    await session.send_client_content(
      turns={"role": "user", "parts": [{"text": prompt}]}, turn_complete=True
    )

    async for response in session.receive():
      if text:=response.text:
        display.display(display.Markdown(text))
        continue

      server_content = response.server_content
      if server_content is not None:
        handle_server_content(server_content)
        continue

      tool_call = response.tool_call
      if tool_call is not None:
        await handle_tool_call(session, tool_call)
        # return await handle_tool_call(session, tool_call)


### **Hello Gemini 2.0**

In [21]:
await run(prompt="Hello?", tools=None)

Hello?

-------------------------------

Hello there

! How can I help you today?


### **Simple Function Calling**

In [22]:
# Function definition
get_current_weather_info = {
        "name": "get_current_weather",
        "description": "Get the current weather of a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use"
                }
            },
            "required": ["location"]
        }
    }

In [23]:
prompt = "What's the weather of Karachi, Pakistan?"

tools = [
    {'function_declarations': [get_current_weather_info]}
]

await run(prompt, tools=tools)


What's the weather of Karachi, Pakistan?

-------------------------------

-------------------------------

``` python
print(default_api.get_current_weather(location="Karachi, Pakistan"))

```

-------------------------------

...

```{'temperature': '33C', 'comments': 'The current weather of Karachi, Pakistan is Sunny and there is no chance of rain today.'}
```

-------------------------------

It

's 33C and sunny in Karachi, Pakistan. There is no chance of rain

 today.


### **Single Turn Function Calling**

In [29]:
start_music = {
    "name": "start_music",
    "description": "Play some music matching the specified parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "energetic": {
                "type": "boolean",
                "description": "Whether the music is energetic or not.",
            },
            "loud": {
                "type": "boolean",
                "description": "Whether the music is loud or not.",
            },
        },
        "required": ["energetic", "loud"],
    },
}

dim_lights = {
    "name": "dim_lights",
    "description": "Dim the lights.",
    "parameters": {
        "type": "object",
        "properties": {
            "brightness": {
                "type": "number",
                "description": "The brightness of the lights, 0.0 is off, 1.0 is full.",
            }
        },
        "required": ["brightness"],
    },
}

In [30]:
prompt = "Turn this place into a party!"

tools = [
    {'function_declarations': [start_music, dim_lights]}
]

# prompt = 'I want moderate lights not too bright'

# prompt = 'Play some music loud but not energetic!'

# prompt = "Turn on the AC"

await run(prompt, tools=tools)

Turn this place into a party!

-------------------------------

Alright

! I can help you with that. I can play some music and dim the lights.

 Should I play loud and energetic music?


### **Parallel Exceution Example:**

In [31]:
prompt = 'Hey, can you write run some python code to check if the string racecar is Palindrome, wait 10s and then turn off the lights?'

tools = [
    {'code_execution': {}},
    {'function_declarations': [dim_lights]}
]

await run(prompt, tools=tools)

Hey, can you write run some python code to check if the string racecar is Palindrome, wait 10s and then turn off the lights?

-------------------------------

-------------------------------

``` python
import time

def is_palindrome(s):
  return s == s[::-1]

word = "racecar"
if is_palindrome(word):
  print(f"{word} is a palindrome")
else:
  print(f"{word} is not a palindrome")

time.sleep(10)
print("Turning off the lights.")
default_api.dim_lights(brightness=0.0)

```

-------------------------------

...

```racecar is a palindrome
Turning off the lights.
```

-------------------------------

OK

. The word "racecar" is a palindrome. I have waited 10 seconds

 and turned off the lights.


### **Compositional Function Calling**

In [ ]:
meeting_tools = [
    {
        "function_declarations": [
            {
                "name": "check_calendar",
                "description": "Check calendar for availability",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                            "description": "The date to check in YYYY-MM-DD format"
                        },
                        "start_time": {
                            "type": "string",
                            "description": "The start time in HH:MM format"
                        },
                        "end_time": {
                            "type": "string",
                            "description": "The end time in HH:MM format"
                        }
                    },
                    "required": ["date", "start_time", "end_time"]
                }
            },
            {
                "name": "create_meeting",
                "description": "Create a calendar event",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "The title of the meeting"
                        },
                        "date": {
                            "type": "string",
                            "description": "The date of the meeting in YYYY-MM-DD format"
                        },
                        "start_time": {
                            "type": "string",
                            "description": "The start time in HH:MM format"
                        },
                        "end_time": {
                            "type": "string",
                            "description": "The end time in HH:MM format"
                        },
                        "attendees": {
                            "type": "array",
                            "items": {
                                "type": "string"
                            },
                            "description": "List of email addresses for attendees"
                        }
                    },
                    "required": ["title", "date", "start_time", "end_time"]
                }
            },
            {
                "name": "send_email",
                "description": "Send an email",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "array",
                            "items": {
                                "type": "string"
                            },
                            "description": "List of email addresses to send to"
                        },
                        "subject": {
                            "type": "string",
                            "description": "The subject of the email"
                        },
                        "body": {
                            "type": "string",
                            "description": "The body of the email"
                        },
                    },
                    "required": ["to", "subject", "body"]
                }
            }
        ]
    }
]

In [ ]:
# Step 1: Check calendar availability
check_calendar_prompt = """
Check if the calendar is available on 2025-05-12 from 2pm to 3pm.
"""
res = await run(check_calendar_prompt, modality="TEXT", tools=meeting_tools)


# Step 2: Create meeting
create_meeting_prompt = """
    The calendar is available on 2025-05-12 from 2pm to 3pm.
    Please create a project review meeting with attendees test1@gmail.com and test2@gmail.com.
    """
calender_res = await run(create_meeting_prompt, modality="TEXT", tools=meeting_tools)


# Step 3: Send email notifications
send_email_prompt = """
        Please send an email to test1@gmail.com and test2@gmail.com with subject of 'Project Review Meeting'
        and description of 'Please join us for the project review meeting.'
        Attach the project review meeting details i.e. 2025-05-12 from 2pm to 3pm and then end the email with a personalized closing that says 'Regards, Hassam (GDG Kolachi Lead)
        """
await run(send_email_prompt, modality="TEXT", tools=meeting_tools)




In [ ]:
# Extended Version

# # Step 1: Check calendar availability
# check_calendar_prompt = """
# Check if the calendar is available on 2025-05-12 from 2pm to 3pm.
# """
# res = await run(check_calendar_prompt, modality="TEXT", tools=meeting_tools)

# if(res[0].response['available']):
#     # Step 2: Create meeting
#     create_meeting_prompt = """
#     The calendar is available on 2025-05-12 from 2pm to 3pm.
#     Please create a project review meeting with attendees test1@gmail.com and test2@gmail.com.
#     """
#     calender_res = await run(create_meeting_prompt, modality="TEXT", tools=meeting_tools)

#     if(calender_res[0].response['created']):
#       # Step 3: Send email notifications
#       send_email_prompt = """
#         A project review meeting has been created for 2025-05-12 at 2pm.
#         Please send an email to test1@gmail.com and test2@gmail.com with subject of 'Project Review Meeting'
#         and description of 'Please join us for the project review meeting.'
#         End the email with a personalized closing that says 'Regards, Hassam (GDG Kolachi Lead)
#         """
#       await run(send_email_prompt, modality="TEXT", tools=meeting_tools)




### **Multi Tool Example:**

In [ ]:

prompt = """\
  Hey, I need you to do three things for me.

  1. Write code in JavaScript language to check whether a number is even or odd.
  2. Then use google search to lookup unformation about the GDG Kolachi and its recent events order by date in an descending order?
  3. Turn on the lights

  Thanks!
  """

tools = [
    {'google_search': {}},
    {'code_execution': {}},
    {'function_declarations': [dim_lights]}
]

await run(prompt, tools=tools)

  Hey, I need you to do three things for me.

  1. Write code in JavaScript language to check whether a number is even or odd.
  2. Then use google search to lookup unformation about the GDG Kolachi and its recent events order by date in an descending order?
  3. Turn on the lights

  Thanks!
  

-------------------------------

Okay

, I can help with that. Here's the breakdown:

1.

  **JavaScript Code:** I will provide the JavaScript code to check if a number is even or odd.
2.  **Google Search:** I will use the search tool

...

``` python
default_api.dim_lights(brightness=1.0)

```

-------------------------------

✅ Function result: {'reslt': 'ok'}

 Response >>>  [FunctionResponse(will_continue=None, scheduling=None, id='function-call-8265470664609986522', name='dim_lights', response={'reslt': 'ok'})]


```tool_outputs


{}
```

### **Google Developer, can you write a custom function that can Turn On/Off the AC at Hive ?**